# Analisis Data Banjir Menggunakan Spark SQL

Notebook ini menggunakan Spark SQL untuk menganalisis data banjir dan menemukan insight penting dari data.

In [ ]:
# Import library yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Mengatur style plot
plt.style.use('ggplot')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Membuat Spark Session dengan dukungan untuk Hive
spark = SparkSession.builder \
    .appName("Analisis Data Banjir") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .enableHiveSupport() \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark master: {spark.sparkContext.master}")

In [ ]:
# Mendefinisikan schema untuk data CSV
schema = StructType([
    StructField("Tanggal", StringType(), True),
    StructField("Provinsi", StringType(), True),
    StructField("Nilai_Server", DoubleType(), True),
    StructField("Nilai_Kartu", DoubleType(), True),
    StructField("Nilai_Registered", DoubleType(), True),
    StructField("Nilai_Unregistered", DoubleType(), True)
])

# Coba membaca file CSV dari berbagai lokasi
file_paths = [
    "DataFixABD.csv",
    "../data/DataFixABD.csv",
    "/home/jovyan/work/DataFixABD.csv",
    "/home/jovyan/data/DataFixABD.csv"
]

df = None
for path in file_paths:
    try:
        print(f"Mencoba membaca dari: {path}")
        df = spark.read.csv(path, header=True, schema=schema)
        # Force evaluation
        count = df.count()
        print(f"Berhasil memuat {count} baris dari {path}")
        break
    except Exception as e:
        print(f"Error: {str(e)}")
        
if df is None:
    raise Exception("Tidak dapat memuat file data dari lokasi manapun")

In [ ]:
# Pra-pemrosesan data
df = df.withColumn("tanggal", to_date(col("Tanggal")))
df = df.withColumn("tahun", year(col("tanggal")))
df = df.withColumn("bulan", month(col("tanggal")))
df = df.withColumn("nama_bulan", date_format(col("tanggal"), "MMMM"))
df = df.withColumn("hari", dayofmonth(col("tanggal")))
df = df.withColumn("hari_dalam_minggu", date_format(col("tanggal"), "EEEE"))

# Menampilkan skema
print("Schema data:")
df.printSchema()

# Menampilkan contoh data
print("\nContoh data:")
df.show(5)

In [ ]:
# Mendaftarkan sebagai tabel/view untuk query SQL
df.createOrReplaceTempView("flood_data")

# Informasi dasar tentang dataset
print("Jumlah total baris:", df.count())
print("Statistik deskriptif:")
df.describe().show()

## Analisis Data Menggunakan Spark SQL

Sekarang kita akan menggunakan SQL untuk menganalisis data dan menemukan insight penting.

In [ ]:
# 1. Analisis Rata-rata Nilai Per Bulan
query = """
SELECT 
    tahun, 
    bulan,
    nama_bulan,
    ROUND(AVG(Nilai_Server), 4) as avg_server,
    ROUND(AVG(Nilai_Kartu), 4) as avg_kartu,
    ROUND(AVG(Nilai_Registered), 4) as avg_registered,
    ROUND(AVG(Nilai_Unregistered), 4) as avg_unregistered
FROM flood_data
GROUP BY tahun, bulan, nama_bulan
ORDER BY tahun, bulan
"""

monthly_avg = spark.sql(query)
print("Rata-rata nilai per bulan:")
monthly_avg.show(20)

# Konversi ke Pandas untuk visualisasi
monthly_avg_pd = monthly_avg.toPandas()

# Visualisasi data bulanan
plt.figure(figsize=(14, 8))
x = range(len(monthly_avg_pd))
month_labels = [f"{row.nama_bulan} {row.tahun}" for _, row in monthly_avg_pd.iterrows()]

plt.plot(x, monthly_avg_pd['avg_server'], marker='o', linewidth=2, label='Server')
plt.plot(x, monthly_avg_pd['avg_kartu'], marker='s', linewidth=2, label='Kartu')
plt.plot(x, monthly_avg_pd['avg_registered'], marker='^', linewidth=2, label='Registered')
plt.plot(x, monthly_avg_pd['avg_unregistered'], marker='d', linewidth=2, label='Unregistered')

plt.xticks(x, month_labels, rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.legend()
plt.title('Rata-rata Nilai Banjir per Bulan', fontsize=16)
plt.ylabel('Nilai Rata-rata')
plt.tight_layout()
plt.show()

In [ ]:
# 2. Identifikasi hari-hari dengan nilai banjir tertinggi
query = """
SELECT 
    tanggal,
    Provinsi,
    Nilai_Server,
    Nilai_Kartu,
    Nilai_Registered,
    Nilai_Unregistered
FROM flood_data
ORDER BY Nilai_Server DESC
LIMIT 10
"""

highest_floods = spark.sql(query)
print("10 hari dengan nilai banjir tertinggi:")
highest_floods.show()

# Visualisasi data tertinggi
highest_floods_pd = highest_floods.toPandas()
highest_floods_pd['tanggal'] = pd.to_datetime(highest_floods_pd['tanggal'])
highest_floods_pd = highest_floods_pd.sort_values('tanggal')

plt.figure(figsize=(14, 8))
x = range(len(highest_floods_pd))
date_labels = [d.strftime('%Y-%m-%d') for d in highest_floods_pd['tanggal']]

bar_width = 0.2
plt.bar([i - 1.5*bar_width for i in x], highest_floods_pd['Nilai_Server'], width=bar_width, label='Server', color='#1f77b4')
plt.bar([i - 0.5*bar_width for i in x], highest_floods_pd['Nilai_Kartu'], width=bar_width, label='Kartu', color='#ff7f0e')
plt.bar([i + 0.5*bar_width for i in x], highest_floods_pd['Nilai_Registered'], width=bar_width, label='Registered', color='#2ca02c')
plt.bar([i + 1.5*bar_width for i in x], highest_floods_pd['Nilai_Unregistered'], width=bar_width, label='Unregistered', color='#d62728')

plt.xticks(x, date_labels, rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.legend()
plt.title('10 Hari dengan Nilai Banjir Tertinggi', fontsize=16)
plt.ylabel('Nilai Banjir')
plt.tight_layout()
plt.show()

In [ ]:
# 3. Analisis tren mingguan - apakah ada hari dalam minggu yang konsisten memiliki nilai banjir lebih tinggi?
query = """
SELECT 
    hari_dalam_minggu,
    ROUND(AVG(Nilai_Server), 4) as avg_server,
    ROUND(AVG(Nilai_Kartu), 4) as avg_kartu,
    ROUND(AVG(Nilai_Registered), 4) as avg_registered,
    ROUND(AVG(Nilai_Unregistered), 4) as avg_unregistered,
    COUNT(*) as jumlah_hari
FROM flood_data
GROUP BY hari_dalam_minggu
ORDER BY 
    CASE 
        WHEN hari_dalam_minggu = 'Monday' THEN 1
        WHEN hari_dalam_minggu = 'Tuesday' THEN 2
        WHEN hari_dalam_minggu = 'Wednesday' THEN 3
        WHEN hari_dalam_minggu = 'Thursday' THEN 4
        WHEN hari_dalam_minggu = 'Friday' THEN 5
        WHEN hari_dalam_minggu = 'Saturday' THEN 6
        WHEN hari_dalam_minggu = 'Sunday' THEN 7
    END
"""

weekly_pattern = spark.sql(query)
print("Pola mingguan nilai banjir:")
weekly_pattern.show()

# Visualisasi pola mingguan
weekly_pattern_pd = weekly_pattern.toPandas()

fig, ax = plt.subplots(figsize=(12, 8))
bar_width = 0.2
x = np.arange(len(weekly_pattern_pd))

ax.bar(x - 1.5*bar_width, weekly_pattern_pd['avg_server'], width=bar_width, label='Server', color='#1f77b4')
ax.bar(x - 0.5*bar_width, weekly_pattern_pd['avg_kartu'], width=bar_width, label='Kartu', color='#ff7f0e')
ax.bar(x + 0.5*bar_width, weekly_pattern_pd['avg_registered'], width=bar_width, label='Registered', color='#2ca02c')
ax.bar(x + 1.5*bar_width, weekly_pattern_pd['avg_unregistered'], width=bar_width, label='Unregistered', color='#d62728')

ax.set_xticks(x)
ax.set_xticklabels(weekly_pattern_pd['hari_dalam_minggu'])
ax.set_ylabel('Rata-rata Nilai Banjir')
ax.set_title('Pola Mingguan Nilai Banjir', fontsize=16)
ax.grid(True, alpha=0.3, axis='y')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 4. Analisis nilai secara statistik - distribusi dan outlier
# Hitung percentile untuk setiap nilai
query = """
SELECT
    'Nilai_Server' as metric,
    MIN(Nilai_Server) as min,
    PERCENTILE_APPROX(Nilai_Server, 0.25) as p25,
    PERCENTILE_APPROX(Nilai_Server, 0.5) as median,
    PERCENTILE_APPROX(Nilai_Server, 0.75) as p75,
    MAX(Nilai_Server) as max,
    AVG(Nilai_Server) as mean,
    STDDEV(Nilai_Server) as stddev
FROM flood_data
UNION
SELECT
    'Nilai_Kartu' as metric,
    MIN(Nilai_Kartu) as min,
    PERCENTILE_APPROX(Nilai_Kartu, 0.25) as p25,
    PERCENTILE_APPROX(Nilai_Kartu, 0.5) as median,
    PERCENTILE_APPROX(Nilai_Kartu, 0.75) as p75,
    MAX(Nilai_Kartu) as max,
    AVG(Nilai_Kartu) as mean,
    STDDEV(Nilai_Kartu) as stddev
FROM flood_data
UNION
SELECT
    'Nilai_Registered' as metric,
    MIN(Nilai_Registered) as min,
    PERCENTILE_APPROX(Nilai_Registered, 0.25) as p25,
    PERCENTILE_APPROX(Nilai_Registered, 0.5) as median,
    PERCENTILE_APPROX(Nilai_Registered, 0.75) as p75,
    MAX(Nilai_Registered) as max,
    AVG(Nilai_Registered) as mean,
    STDDEV(Nilai_Registered) as stddev
FROM flood_data
UNION
SELECT
    'Nilai_Unregistered' as metric,
    MIN(Nilai_Unregistered) as min,
    PERCENTILE_APPROX(Nilai_Unregistered, 0.25) as p25,
    PERCENTILE_APPROX(Nilai_Unregistered, 0.5) as median,
    PERCENTILE_APPROX(Nilai_Unregistered, 0.75) as p75,
    MAX(Nilai_Unregistered) as max,
    AVG(Nilai_Unregistered) as mean,
    STDDEV(Nilai_Unregistered) as stddev
FROM flood_data
"""

statistics = spark.sql(query)
print("Statistik untuk semua metrik:")
statistics.show()

# Convert ke pandas untuk visualisasi boxplot
df_pd = df.select("Nilai_Server", "Nilai_Kartu", "Nilai_Registered", "Nilai_Unregistered").toPandas()

plt.figure(figsize=(14, 8))
boxprops = dict(linestyle='-', linewidth=2)
medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
whiskerprops = dict(linestyle='--', linewidth=1.5)

sns.boxplot(data=df_pd, palette="Set3", boxprops=boxprops, medianprops=medianprops, whiskerprops=whiskerprops)
plt.title('Distribusi Nilai Banjir', fontsize=16)
plt.grid(True, alpha=0.3, axis='y')
plt.ylabel('Nilai')
plt.tight_layout()
plt.show()

In [ ]:
# 5. Analisis korelasi antar metrik
# Hitung korelasi menggunakan Spark
print("Korelasi antar metrik:")
print(f"Korelasi Nilai_Server dan Nilai_Kartu: {df.stat.corr('Nilai_Server', 'Nilai_Kartu'):.4f}")
print(f"Korelasi Nilai_Server dan Nilai_Registered: {df.stat.corr('Nilai_Server', 'Nilai_Registered'):.4f}")
print(f"Korelasi Nilai_Server dan Nilai_Unregistered: {df.stat.corr('Nilai_Server', 'Nilai_Unregistered'):.4f}")
print(f"Korelasi Nilai_Kartu dan Nilai_Registered: {df.stat.corr('Nilai_Kartu', 'Nilai_Registered'):.4f}")
print(f"Korelasi Nilai_Kartu dan Nilai_Unregistered: {df.stat.corr('Nilai_Kartu', 'Nilai_Unregistered'):.4f}")
print(f"Korelasi Nilai_Registered dan Nilai_Unregistered: {df.stat.corr('Nilai_Registered', 'Nilai_Unregistered'):.4f}")

# Visualisasi korelasi dengan heatmap
corr_matrix = df_pd.corr()
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt='.4f')

plt.title('Korelasi antar Metrik Banjir', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# 6. Identifikasi bulan dengan nilai tertinggi dan terendah
query_highest = """
SELECT 
    tahun,
    bulan,
    nama_bulan,
    ROUND(AVG(Nilai_Server), 4) as avg_server
FROM flood_data
GROUP BY tahun, bulan, nama_bulan
ORDER BY avg_server DESC
LIMIT 3
"""

query_lowest = """
SELECT 
    tahun,
    bulan,
    nama_bulan,
    ROUND(AVG(Nilai_Server), 4) as avg_server
FROM flood_data
GROUP BY tahun, bulan, nama_bulan
ORDER BY avg_server ASC
LIMIT 3
"""

highest_months = spark.sql(query_highest)
lowest_months = spark.sql(query_lowest)

print("Bulan dengan rata-rata nilai banjir tertinggi:")
highest_months.show()

print("Bulan dengan rata-rata nilai banjir terendah:")
lowest_months.show()

# Visualisasi bulan tertinggi dan terendah
highest_pd = highest_months.toPandas()
lowest_pd = lowest_months.toPandas()
extremes = pd.concat([highest_pd, lowest_pd])
extremes['month_year'] = extremes.apply(lambda row: f"{row['nama_bulan']} {row['tahun']}", axis=1)
extremes['kategori'] = ['Tertinggi'] * len(highest_pd) + ['Terendah'] * len(lowest_pd)

plt.figure(figsize=(12, 8))
colors = {'Tertinggi': '#d62728', 'Terendah': '#1f77b4'}
ax = sns.barplot(x='month_year', y='avg_server', hue='kategori', data=extremes, palette=colors)

plt.title('Bulan dengan Nilai Banjir Tertinggi dan Terendah', fontsize=16)
plt.xlabel('Bulan-Tahun')
plt.ylabel('Rata-rata Nilai Server')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

for i, p in enumerate(ax.patches):
    ax.annotate(f'{p.get_height():.4f}', 
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'bottom',
                xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.show()

In [ ]:
# 7. Analisis tren temporal - apakah nilai banjir meningkat atau menurun dari waktu ke waktu?
query = """
SELECT
    tanggal,
    Nilai_Server,
    Nilai_Kartu,
    Nilai_Registered,
    Nilai_Unregistered
FROM flood_data
ORDER BY tanggal
"""

time_series = spark.sql(query)
time_series_pd = time_series.toPandas()
time_series_pd['tanggal'] = pd.to_datetime(time_series_pd['tanggal'])

# Plot time series dengan trend line
plt.figure(figsize=(16, 10))

# Plot data asli
plt.plot(time_series_pd['tanggal'], time_series_pd['Nilai_Server'], 'o-', alpha=0.5, label='Server (data asli)')

# Tambahkan trend line menggunakan rolling average
window = 30  # 30-day rolling average
time_series_pd['Server_MA'] = time_series_pd['Nilai_Server'].rolling(window=window).mean()
plt.plot(time_series_pd['tanggal'], time_series_pd['Server_MA'], 'r-', linewidth=3, label=f'Server (MA {window} hari)')

# Tambahkan regresi linear untuk melihat tren jangka panjang
from sklearn.linear_model import LinearRegression
import matplotlib.dates as mdates

# Convert dates to ordinal numbers for regression
X = np.array([(d - pd.Timestamp("1970-01-01")) // pd.Timedelta("1D") for d in time_series_pd['tanggal']]).reshape(-1, 1)
y = time_series_pd['Nilai_Server']

# Fit regression model
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)

# Plot regression line
plt.plot(time_series_pd['tanggal'], y_pred, 'g--', linewidth=2, label='Linear trend')

# Calculate slope direction and magnitude (annual rate of change)
slope = model.coef_[0]
annual_change = slope * 365  # change per year
direction = "meningkat" if slope > 0 else "menurun"

# Add annotation about trend
trend_text = f"Tren: {direction} sebesar {abs(annual_change):.4f} per tahun"
plt.annotate(trend_text, xy=(0.05, 0.95), xycoords='axes fraction',
             bbox=dict(boxstyle="round,pad=0.5", fc="yellow", alpha=0.8),
             fontsize=12, ha='left', va='top')

plt.title('Tren Nilai Banjir Selama Periode Data', fontsize=16)
plt.xlabel('Tanggal')
plt.ylabel('Nilai Banjir (Server)')
plt.grid(True, alpha=0.3)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

print(f"Analisis Tren Linear:")
print(f"Koefisien (slope): {slope:.6f} per hari")
print(f"Perubahan tahunan: {annual_change:.4f} per tahun")
print(f"Nilai banjir cenderung {direction} seiring waktu")

In [ ]:
# 8. Uji hipotesis - apakah perbedaan nilai antar metrik signifikan?
from scipy import stats

# Persiapkan data untuk tes statistik
server_values = df_pd['Nilai_Server']
kartu_values = df_pd['Nilai_Kartu']
registered_values = df_pd['Nilai_Registered']
unregistered_values = df_pd['Nilai_Unregistered']

# One-way ANOVA untuk semua grup
anova_result = stats.f_oneway(server_values, kartu_values, registered_values, unregistered_values)
print(f"Analisis Variansi (ANOVA):")
print(f"F-statistic: {anova_result.statistic:.4f}")
print(f"p-value: {anova_result.pvalue:.8f}")
print(f"Kesimpulan: {'Perbedaan signifikan' if anova_result.pvalue < 0.05 else 'Tidak ada perbedaan signifikan'} antar metrik")

# Paired t-test untuk antar pasangan metrik
print("\nUji T Berpasangan:")
tests = [
    ('Server vs Kartu', stats.ttest_rel(server_values, kartu_values)),
    ('Server vs Registered', stats.ttest_rel(server_values, registered_values)),
    ('Server vs Unregistered', stats.ttest_rel(server_values, unregistered_values)),
    ('Kartu vs Registered', stats.ttest_rel(kartu_values, registered_values)),
    ('Kartu vs Unregistered', stats.ttest_rel(kartu_values, unregistered_values)),
    ('Registered vs Unregistered', stats.ttest_rel(registered_values, unregistered_values))
]

for name, result in tests:
    print(f"{name}: t={result.statistic:.4f}, p={result.pvalue:.8f}, "
          f"{'Signifikan' if result.pvalue < 0.05 else 'Tidak signifikan'}")

In [ ]:
# 9. Identifikasi outlier dan hari-hari tidak biasa
from scipy import stats

# Hitung z-score untuk deteksi outlier
df_pd['z_score_server'] = stats.zscore(df_pd['Nilai_Server'])
outliers = df_pd[abs(df_pd['z_score_server']) > 3]  # threshold 3 sigma

# Dapatkan tanggal untuk outlier yang terdeteksi
time_series_pd['z_score'] = stats.zscore(time_series_pd['Nilai_Server'])
outlier_dates = time_series_pd[abs(time_series_pd['z_score']) > 3]

print(f"Terdeteksi {len(outliers)} outlier dengan threshold z-score > 3")
if not outlier_dates.empty:
    print("\nTanggal-tanggal dengan nilai banjir tidak biasa (outlier):")
    display(outlier_dates[['tanggal', 'Nilai_Server', 'z_score']].sort_values(by='z_score', ascending=False))
    
    # Visualisasi outlier dalam time series
    plt.figure(figsize=(16, 8))
    plt.plot(time_series_pd['tanggal'], time_series_pd['Nilai_Server'], 'b-', alpha=0.7, label='Nilai Server')
    plt.scatter(outlier_dates['tanggal'], outlier_dates['Nilai_Server'], 
                color='red', s=50, label='Outlier')
    
    for i, row in outlier_dates.iterrows():
        plt.annotate(row['tanggal'].strftime('%Y-%m-%d'), 
                     xy=(mdates.date2num(row['tanggal']), row['Nilai_Server']),
                     xytext=(10, 20),
                     textcoords='offset points',
                     arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))
    
    plt.title('Deteksi Outlier pada Time Series Nilai Banjir', fontsize=16)
    plt.xlabel('Tanggal')
    plt.ylabel('Nilai Server')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Tidak ditemukan outlier yang signifikan dalam data.")

## Kesimpulan dan Insight

Berdasarkan analisis yang telah dilakukan, berikut beberapa insight utama yang dapat disimpulkan:

1. **Tren Temporal:** 
   - Apakah nilai banjir cenderung meningkat atau menurun dari waktu ke waktu?
   - Periode mana yang memiliki nilai tertinggi dan apakah ada pola musiman?

2. **Pola Mingguan:**
   - Apakah ada hari-hari tertentu dalam seminggu yang konsisten memiliki nilai banjir lebih tinggi?
   - Insight ini dapat membantu untuk antisipasi dan persiapan pada hari-hari rawan.

3. **Korelasi antar Metrik:**
   - Hubungan yang kuat antara nilai server, kartu, registered, dan unregistered, yang menunjukkan konsistensi pengukuran.
   - Perbedaan signifikan antar metrik menunjukkan bahwa tiap metrik mengukur aspek yang berbeda dari fenomena banjir.

4. **Periode Kritis:**
   - Bulan-bulan dengan nilai tertinggi menunjukkan periode kritis yang membutuhkan kewaspadaan ekstra.
   - Identifikasi outlier menunjukkan kejadian-kejadian ekstrem yang perlu investigasi lebih lanjut.

5. **Rekomendasi:**
   - Berdasarkan analisis ini, dapat disusun rekomendasi untuk manajemen bencana dan sistem peringatan dini banjir.

In [ ]:
# Hapus spark session
spark.stop()
print("Analisis selesai, Spark session ditutup.")